In [2]:
from general_functions import *
place = 'Tel Aviv'
my_preprocessing = Preprocessing(place,is_test=True)
data_folder, data_test = my_preprocessing.create_folder()

Data folder: C:\Users\18059\OneDrive - ariel.ac.il\Current_research\SOD\Code\places/Tel_Aviv
Data folder: C:\Users\18059\OneDrive - ariel.ac.il\Current_research\SOD\Code\places/Tel_Aviv/test


In [ ]:

data_test_topology = f'{data_test}/topology'
new_network  = gpd.read_file(f'{data_folder}/simp.shp')
num = 0
new_gpd = new_network.copy()
obj_intersection = Intersection(new_gpd, num)
obj_intersection.intersection_network()
obj_intersection.update_names(new_gpd)
obj_intersection.my_network.to_file(f'{data_test_topology}/topology_0.shp')
line_name = 'line_name'
if my_preprocessing.is_junction:
    print('Update roundabout')
    exist_data = obj_intersection.my_network.reset_index().reset_index(names=line_name)
    
    my_roundabout = Roundabout(exist_data, gpd.read_file(f'{data_test}/preprocessing/roundabout.shp'))
    deadend_lines, deadend_pnts = my_roundabout.deadend()

    # update the current network
    change_geo = my_roundabout.my_spatial_join(deadend_lines, deadend_pnts, line_name)
    my_roundabout.update_the_current_network(change_geo)

    my_roundabout.network.drop_duplicates(subset=line_name, inplace=True)
    # Improve roundabout
    # First buffer around centroid
    centr_name = 'centr_name'
    buffer_around_centroid = my_roundabout.centroid['geometry'].buffer(cap_style=1, distance=30)

    # s_join between buffer to lines (reset index to retain the original centroid name which can apper more than one in the results). always stay with data you need and with understandable name
    roundabout_with_lines = \
        gpd.sjoin(left_df=GeoDataFrame(geometry=buffer_around_centroid, crs=project_crs).reset_index(),
                  right_df=my_roundabout.network[['geometry', line_name]]).drop_duplicates(
            subset=['index', line_name]).rename(columns={"index": centr_name})[['geometry', line_name, centr_name]]

    # To facilitate the searching process
    my_roundabout.network.set_index(line_name, inplace=True)
    # To facilitate easy access to point centroid geometry data, it is advisable to store the information in an object that provides efficient retrieval.
    pnt_centroid_temp = my_roundabout.centroid['geometry']
    #  Group the data by centroid
    for center_line in roundabout_with_lines.groupby(centr_name):
        #  Iterate over each group after performing a groupby() operation
        for center in center_line[1].itertuples():
            # Find the line that connects to the current centroid and obtain its vertices
            line_to_test = my_roundabout.network.loc[center[2]]
            vertices_line = list(line_to_test['geometry'].coords)
            pnt_test = [vertices_line[0], vertices_line[-1]]
            # To determine if the current line is already connected to the current centroid.
            is_connected = my_roundabout.centroid[
                my_roundabout.centroid['geometry'].isin([Point(pnt_test[0]), Point(pnt_test[-1])])]
            if len(is_connected) > 0 and center[3] in is_connected['name']:
                continue

            if len(vertices_line) == 2:
                vertices_line.insert(1, pnt_centroid_temp[center[3]])
            else:
                my_list = [pnt_centroid_temp[center[3]].distance(Point(temp)) for temp in vertices_line]
                # Find the minimum index
                min_index = min(range(len(my_list)), key=my_list.__getitem__)
                if min_index == 0:
                    vertices_line.insert(0, pnt_centroid_temp[center[3]])
                elif min_index == len(my_list) - 1:
                    vertices_line.append(pnt_centroid_temp[center[3]])
                else:
                    vertices_line[min_index] = pnt_centroid_temp[center[3]]
            new_geo = LineString(vertices_line)
            my_roundabout.network.at[center[2], 'geometry'] = new_geo

    new_network1 = my_roundabout.network.reset_index()
    new_network1.drop(columns='index', inplace=True)


    # Function to remove self-intersecting LineString geometries
    def remove_self_intersecting(line):
        return line.is_simple


    # Apply the function to filter out self-intersecting geometries
    new_network2 = new_network1[new_network1['geometry'].apply(remove_self_intersecting)]
else:
    new_network2 = obj_intersection.my_network.reset_index()
    

extend_lines_f = extend_lines(new_network2, 100)
extend_lines_f['length'] = extend_lines_f.length
obj_intersection_1 = Intersection(extend_lines_f.copy(), 1)
obj_intersection_1.intersection_network()
obj_intersection_1.my_network.rename(columns={'str_name': 'name'},
                                     inplace=True)  # 'str_name' become 'str to be compatible with other previous networks
obj_intersection_1.update_names(org_gpd=extend_lines_f.copy().rename(columns={'str_name': 'name'}))



# Clear short segments
final2 = EnvEntity(obj_intersection_1.my_network.reset_index())
final2.update_the_current_network(final2.get_deadend_gdf(delete_short=100))
final2.network.drop(columns=[line_name, 'bearing']).to_file(f'{data_test}/network_test.shp')


Update topology


100%|██████████| 884/884 [00:19<00:00, 46.28it/s]


Update roundabout
Update topology


 74%|███████▍  | 18915/25618 [00:13<00:05, 1302.03it/s]

5671,4219:GEOMETRYCOLLECTION (LINESTRING (3874113.596534388 3769820.5046853954, 3874107.23156703 3769818.2034434094), POINT (3874112.61501554 3769817.5204950683))


 76%|███████▌  | 19479/25618 [00:14<00:04, 1373.48it/s]

4219,5671:GEOMETRYCOLLECTION (LINESTRING (3874113.596534388 3769820.5046853954, 3874107.23156703 3769818.2034434094), POINT (3874112.61501554 3769817.5204950683))


 87%|████████▋ | 22162/25618 [00:16<00:02, 1394.04it/s]

5545,5405:GEOMETRYCOLLECTION (LINESTRING (3870087.7663300117 3768727.4177614003, 3870098.701060125 3768733.389578613), POINT (3870094.2434388874 3768734.605149279))
5405,5545:GEOMETRYCOLLECTION (LINESTRING (3870087.7663300117 3768727.4177614003, 3870098.701060125 3768733.389578613), POINT (3870094.2434388874 3768734.605149279))


100%|██████████| 522/522 [00:16<00:00, 31.76it/s]
